<a href="https://colab.research.google.com/github/thomas-chong/fine-tuning-gemma-with-unsloth/blob/main/live_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Presented by Thomas Chong**
*   **Google Developer Expert in AI**
*   **AI Research Engineer at Beever AI**

<a href="https://www.linkedin.com/in/chongcht/" target="_blank"><img src="https://img.shields.io/badge/LinkedIn-0077B5?style=for-the-badge&logo=linkedin&logoColor=white" alt="LinkedIn"></a>
<a href="https://scholar.google.com/citations?user=aV-Ddp4AAAAJ&hl=en" target="_blank"><img src="https://img.shields.io/badge/Google_Scholar-4285F4?style=for-the-badge&logo=google-scholar&logoColor=white" alt="Google Scholar"></a>
<a href="https://github.com/thomas-chong" target="_blank"><img src="https://img.shields.io/badge/GitHub-181717?style=for-the-badge&logo=github&logoColor=white" alt="GitHub"></a>
<a href="https://thomas-chong.github.io/" target="_blank"><img src="https://img.shields.io/badge/Website-464646?style=for-the-badge&logo=About.me&logoColor=white" alt="Website"></a>


By day, I'm an AI Research Engineer at [Beever AI](https://beever.ai/), a new-established AI research subsidiary in Toronto, by [Votee AI](https://votee.ai/) in Hong Kong. My work involves a mix of in-house R&D for our agentic AI products and publishing research papers through collaborations with external partners.

# Part 1: Introduction & Setup

### **Live Model Fine-Tuning: Customizing Gemma 3 270M in Under 30 Minutes**

<div style="text-align: center;">
  <img src="https://raw.githubusercontent.com/thomas-chong/fine-tuning-gemma-with-unsloth/main/images/devfest_waterloo.png" alt="DevFest Waterloo" style="width: 100%; max-width: 600px;">
</div>

This session provides a comprehensive, live demonstration of a complete fine-tuning workflow for Google's Gemma 3 270M model, executed in under 30 minutes. The primary objective is to illustrate how recent advancements in model architecture, training techniques, and software optimization have made customizing powerful Large Language Models (LLMs) accessible to a broader audience, even in resource-constrained environments.

The traditional approach, full fine-tuning, involves updating every single parameter in the model. This process is computationally intensive, requiring significant GPU memory and extended training times. This notebook will showcase a modern, efficient alternative by strategically combining three key technologies:

1.  **Gemma 3 270M**: A compact, state-of-the-art open model from Google. Its small footprint makes it an ideal candidate for rapid experimentation and deployment on consumer-grade hardware.
2.  **QLoRA (Quantized Low-Rank Adaptation)**: A highly efficient fine-tuning method. QLoRA dramatically reduces memory by quantizing the model's weights to 4-bits and then inserting small, trainable "adapter" layers. We only update these adapters, leaving the millions of base model parameters frozen.
3.  **Unsloth**: An open-source library with optimized CUDA kernels that can make fine-tuning up to 2x faster while reducing memory usage by 60-70%. Unsloth is the critical enabler that makes this sub-30-minute demonstration feasible on a free Google Colab GPU.

This rapid fine-tuning is possible due to the convergence of these three elements: a capable base model, a memory-frugal training technique, and a speed-optimized library.

**Agenda:**
1.  **Prerequisites:** What are Fine-Tuning and LoRA?
2.  **Setup & Installation:** Getting our environment ready.
3.  **Data Generation:** Creating a synthetic dataset with the Gemini Batch API.
4.  **Live Fine-Tuning:** Loading, configuring, and training our model.
5.  **Evaluation:** Was it worth it? A side-by-side comparison with an LLM-as-a-Judge.
6.  **Conclusion & Resources.**


### The Overall Fine-Tuning Pipeline

This diagram illustrates the end-to-end workflow we will follow in this notebook, from generating a synthetic dataset to exporting a locally-runnable model.

```mermaid
graph LR
    subgraph "Part 1: Data Generation"
        A[Start: Define Creative Themes] --> B{Gemini Batch API};
        B --> C[Generate Synthetic Dataset];
    end

    subgraph "Part 2: Model Training"
        C --> D[Load Base Gemma 3 Model];
        D --> E[Inject QLoRA Adapters];
        E --> F(Fine-Tune with Unsloth);
    end

    subgraph "Part 3: Evaluation & Export"
        F --> G{A/B Test: Base vs. Fine-Tuned};
        G --> H[LLM-as-a-Judge Verdict];
        H --> I[Save LoRA Adapters];
        I --> J[Export to GGUF for Local Inference];
        J --> K[End: Ready for LM Studio];
    end

    style F fill:#a2de89,stroke:#333,stroke-width:2px
    style J fill:#f9d77f,stroke:#333,stroke-width:2px
```


### What is Fine-Tuning?

Fine-tuning adapts a general pre-trained model to a specific task using a smaller, focused dataset. It's like teaching a polymath to become a specialist in a particular field.

### What is LoRA (Low-Rank Adaptation)?

Full fine-tuning requires updating all model weights, which is computationally expensive. **LoRA** offers a much more efficient alternative. It freezes the original weights and injects small, trainable "adapter" matrices (A and B) into the model. We only train these adapters, which are a tiny fraction of the total parameters.

<div style="text-align: center;">
  <img src="https://raw.githubusercontent.com/thomas-chong/fine-tuning-gemma-with-unsloth/main/images/what_is_lora.png" alt="LoRA Diagram" style="width: 100%; max-width: 800px;">
</div>

This makes fine-tuning incredibly fast and memory-efficient. But does it compromise on quality?

> #### **Key Insight from "LoRA Without Regret"**
>
> Research shows that LoRA can match the performance of full fine-tuning if configured correctly. One of the most critical, and often overlooked, conditions is **applying LoRA to all possible layers.**
>
> From the paper: *"Attention-only LoRA significantly underperforms MLP-only LoRA, and does not further improve performance on top of LoRA-on-MLP."*
>
> This is because the MLP (feed-forward) layers contain a vast number of parameters compared to the attention layers. By ignoring them, you're leaving most of the model's knowledge untapped and creating a bottleneck for learning. **Therefore, we will target all linear layers in our configuration.**
>
> Link to the blog post: [LoRA Without Regret](https://thinkingmachines.ai/blog/lora/)

<div style="text-align: center;">
  <img src="https://raw.githubusercontent.com/thomas-chong/fine-tuning-gemma-with-unsloth/main/images/lora_without_regret.png" alt="LoRA Diagram" style="width: 100%; max-width: 800px;">
</div>


### Install Dependencies

Install Unsloth from GitHub for the latest patches and pin other libraries for reproducibility.

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

# Install the Google GenAI SDK for using the Gemini API.
%pip install -q -U "google-genai"

### Configure API Keys

To run this notebook, you'll need API keys for both Google Gemini and the Hugging Face Hub. For security, we'll use Colab's built-in secret manager.

1.  Click on the **key icon** (🔑) in the left-hand sidebar.
2.  Create two new secrets:
    *   **Name:** `GOOGLE_API_KEY` -> **Value:** Your Gemini API Key
    *   **Name:** `HF_TOKEN` -> **Value:** Your Hugging Face access token (with `write` permissions)

Once you've added these secrets, the notebook will be able to access them securely.

<div style="text-align: center;">
    <img src="https://raw.githubusercontent.com/thomas-chong/fine-tuning-gemma-with-unsloth/main/images/secret_settings.png" width="600">
</div>


In [ ]:
#@title  Configurations
#@markdown ---
#@markdown ### **Model & Dataset**
model_name = "unsloth/gemma-3-270m-it" #@param {type:"string"}
max_seq_length = 2048 #@param {type:"integer"}
use_pregenerated_dataset = True #@param {type:"boolean"}
#@markdown ---
#@markdown ### **LoRA Parameters**
lora_r = 16 #@param {type:"slider", min:4, max:64, step:4}
#@markdown ---
#@markdown ### **Training Parameters**
max_steps = 150 #@param {type:"integer"}
learning_rate = 2e-4 #@param {type:"number"}
#@markdown ---
#@markdown ### **Hugging Face**
hf_user = "your-username" #@param {type:"string"}
#@markdown ---


In [ ]:
### Import Libraries & Configure Secrets
# Core ML and Unsloth imports
from unsloth import FastModel
import torch
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments
from datasets import load_dataset

# Google Gemini API imports
from google import genai
from google.genai import types
import json
import time
from google.colab import userdata

# Get API keys from Colab secrets
hf_token = userdata.get('HF_TOKEN')
google_api_key = userdata.get('GOOGLE_API_KEY')

# Part 2: Data is Fuel - Synthetic Data Generation


### Bootstrapping Our Dataset with the Gemini Batch API

<div style="text-align: center;">
  <img src="https://raw.githubusercontent.com/thomas-chong/fine-tuning-gemma-with-unsloth/main/images/gemini_batch_api.png" alt="Gemini Batch API" style="width: 100%; max-width: 800px;">
</div>

A common challenge in fine-tuning is the "cold start" problem, where you have a task in mind but lack labeled training examples. **Synthetic data generation** offers a powerful solution, allowing us to use a highly capable LLM to create a labeled dataset from scratch.

This section demonstrates how to use the **Gemini Batch API** to generate our creative writing dataset. This API is specifically designed for high-volume, asynchronous tasks. It processes large numbers of requests in parallel at a 50% cost discount compared to its real-time counterpart, making it an ideal tool for large-scale data generation.

The workflow is as follows:
1. A prompt template is designed to instruct the Gemini model to generate creative stories.
2. These prompts are formatted into a JSON Lines (JSONL) file.
3. The file is submitted as an asynchronous batch job.

Since batch jobs are not instantaneous, this demonstration will show the code for submitting the job and then proceed with a pre-generated set of results to maintain the pace of the live session. This approach showcases a sophisticated data augmentation strategy that enhances the model's generalization capabilities.


In [ ]:
# --- CONFIGURATION ---
HF_DATASET_NAME = f"{hf_user}/synthetic-creative-writing"

if not use_pregenerated_dataset:
    from datasets import Dataset
    # --- Gemini API Setup ---
    # Initialize the client for Batch API usage
    client = genai.Client(api_key=google_api_key)

    # --- Data Generation Pipeline ---
    SYSTEM_PROMPT = "You are a master storyteller. Write a short, imaginative story based on the user's request. The story should be concise and suitable for a general audience. The story should not exceed 2048 tokens."

    STORY_THEMES = [
        "A robot who discovers music for the first time.",
        "A magical library where books come to life.",
        "A detective who solves crimes in a city powered by steam.",
        "Two pen pals from different planets meeting for the first time.",
        "A mischievous forest spirit who plays pranks on hikers.",
        "The last dragon on Earth sharing its wisdom with a young child.",
    ]

    def create_batch_requests(themes, system_prompt):
        """Creates a JSONL file for the Gemini Batch API."""
        jsonl_file_path = 'synthetic_story_requests.jsonl'
        with open(jsonl_file_path, 'w') as f:
            for i, theme in enumerate(themes):
                request = {
                    "key": f"request_{i}",
                    "request": {
                         "systemInstruction": {
                            "parts": [{"text": system_prompt}]
                        },
                        "contents": [
                           {"parts": [{"text": theme}]}
                        ],
                        "generationConfig": {
                            "maxOutputTokens": 2048,
                            "temperature": 0.9,
                        }
                    }
                }
                f.write(json.dumps(request) + '\n')
        return jsonl_file_path

    # Create and run the batch job
    print("Creating batch requests file...")
    requests_file = create_batch_requests(STORY_THEMES * 50, SYSTEM_PROMPT) # Create 300 examples
    print(f"Uploading file: {requests_file}")
    uploaded_file = client.files.upload(
        file=requests_file,
        config=types.UploadFileConfig(display_name='my-batch-requests', mime_type='jsonl')
    )
    batch_job = client.batches.create(
        model="gemini-2.5-flash",
        src=uploaded_file.name,
    )
    print(f"Batch job created: {batch_job.name}. Polling for results...")

    # Polling logic
    while True:
        batch_job = client.batches.get(name=batch_job.name)
        if batch_job.state.name in ('JOB_STATE_SUCCEEDED', 'JOB_STATE_FAILED', 'JOB_STATE_CANCELLED'):
            break
        print(f"Job not finished. Current state: {batch_job.state.name}. Waiting 30 seconds...")
        time.sleep(30)

    print(f"Job finished with state: {batch_job.state.name}")

    if batch_job.state.name == 'JOB_STATE_SUCCEEDED':
        results = []
        result_file_name = batch_job.dest.file_name
        print(f"Results are in file: {result_file_name}")

        file_content_bytes = client.files.download(file=result_file_name)
        file_content = file_content_bytes.decode('utf-8')

        print("Parsing results...")
        for line in file_content.splitlines():
            if line:
                parsed_response = json.loads(line)
                original_request_key = parsed_response.get('key', '')
                try:
                    request_index = int(original_request_key.split('_')[1])
                    prompt = STORY_THEMES[request_index % len(STORY_THEMES)]
                except (IndexError, ValueError):
                    prompt = "Unknown prompt"

                response_text = ""
                try:
                    response_text = parsed_response['response']['candidates'][0]['content']['parts'][0]['text']
                except (KeyError, IndexError):
                    response_text = "Error: Could not parse response."

                results.append({'prompt': prompt, 'response': response_text})

        # Create a Hugging Face dataset
        dataset = Dataset.from_list(results)

        # Optional: Push to Hub
        if hf_user != "your-username":
            try:
                print(f"Pushing dataset to Hugging Face Hub: {HF_DATASET_NAME}")
                dataset.push_to_hub(HF_DATASET_NAME, private=True, token=hf_token)
                print("Dataset pushed successfully.")
                use_pregenerated_dataset = True
            except Exception as e:
                print(f"Failed to push dataset to Hub: {e}")
                print("Proceeding with local dataset.")
        else:
            print("hf_user is not set. Skipping push to Hub and proceeding with local dataset.")

    else:
        if batch_job.state.name == 'JOB_STATE_FAILED':
            print(f"Job failed. Error: {batch_job.error}")
        else:
            print(f"Job did not succeed (State: {batch_job.state.name}).")

        # Fallback to pregenerated dataset if generation fails
        print("Falling back to pre-generated dataset.")
        use_pregenerated_dataset = True


# Part 3: Live Fine-Tuning with Unsloth


In [ ]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    load_in_4bit = True, # Use QLoRA for memory efficiency
)


### Data Formatting for Gemma 3

Instruction-tuned models like Gemma 3 are pre-trained on data formatted with specific "chat templates." These templates use special tokens to delineate conversational turns (e.g., from a user and a model). Adhering to this format during fine-tuning is critical for the model to correctly interpret the task and generate responses in the desired style.

The Gemma 3 chat template follows this structure:

`<bos><start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n{response}<end_of_turn><eos>`

Failing to use the correct template introduces a **distribution shift** between our fine-tuning data and the model's original training data. This mismatch can confuse the model, leading to suboptimal performance. This formatting step is not merely cleaning but a crucial alignment of our custom data with the model's pre-existing conversational conditioning.


In [ ]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template

PREGENERATED_DATSET_NAME = "chongcht/synthetic-creative-writing"

if use_pregenerated_dataset:
    try:
        dataset = load_dataset(PREGENERATED_DATSET_NAME, split="train")
        print(f"Loaded pre-generated dataset from {PREGENERATED_DATSET_NAME}")
        print(f"Dataset columns: {dataset.column_names}")
    except Exception as e:
        print(f"Could not load dataset from {PREGENERATED_DATSET_NAME}. Please ensure it exists and is public.")
        # If pre-generated dataset fails to load, we might need a fallback or to stop.
        # For now, let's assume the synthetic generation in the previous cell will run.
        # If you are running this cell directly, ensure use_pregenerated_dataset is False
        # or the dataset exists and is accessible.


# Format the dataset for Gemma 3
# Ensure the dataset has 'prompt' and 'response' columns or adapt accordingly.
def format_for_gemma(example):
    prompt_text = example.get("prompt", "") # Use .get() for safer access
    response_text = example.get("response", "") # Use .get() for safer access

    # Handle cases where the data is already in a 'conversations' format
    if not prompt_text and "conversations" in example:
        for message in example["conversations"]:
            if message["role"] == "user":
                prompt_text = message["content"]
            elif message["role"] == "assistant":
                response_text = message["content"]

    if not prompt_text or not response_text:
        # Handle cases where prompt or response might be missing
        print(f"Skipping example due to missing prompt or response: {example}")
        return None # Return None to indicate this example should be skipped

    return {
        "conversations": [
            {"role": "system", "content": "You are a master storyteller. Write a short, imaginative story based on the user's request. The story should be concise and suitable for a general audience."},
            {"role": "user", "content": prompt_text},
            {"role": "assistant", "content": response_text},
        ]
    }

# Apply the formatting function and filter out None results
dataset = dataset.map(format_for_gemma, remove_columns=dataset.column_names).filter(lambda x: x is not None)


print("Dataset formatted. Here's an example:")
if len(dataset) > 0:
    print(dataset[0]['conversations'])
else:
    print("No valid examples found in the dataset after formatting.")

# Apply the chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma3",
)

def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

print("\n\nDataset after applying chat template. Here's the first example:")
if len(dataset) > 0:
    print(dataset[0]['text'])
else:
    print("No valid examples found in the dataset after applying chat template.")

### The Fine-Tuning Core: QLoRA with Unsloth

This section constitutes the core of the demonstration: the fine-tuning process itself. Here, QLoRA is employed to adapt the Gemma 3 model. The Unsloth library's `FastModel` class is central to this, automatically applying memory and speed optimizations that abstract away much of the boilerplate code.

A key focus here is the **deliberate and evidence-based selection of LoRA hyperparameters**. The choices for rank (`r`), `lora_alpha`, and `target_modules` are not arbitrary; they are directly informed by empirical findings and best practices to maximize performance.

| Hyperparameter     | Value             | Justification & Source                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|:-------------------|:------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **r (Rank)**       | 16                | The rank determines the capacity of the LoRA adapters. `r = 16` provides a robust balance between expressive power and parameter efficiency for many tasks. While higher ranks can capture more complex patterns, they also increase memory usage and the risk of overfitting. (Source: Unsloth LoRA Guide)                                                                                                                                                                |
| **lora_alpha**     | 32                | The alpha parameter acts as a scalar for the adapter weights. A common and effective heuristic is to set `lora_alpha = 2 * r`. This scaling gives more weight to the fine-tuned adjustments, helping the model learn the new task more effectively. (Source: Unsloth LoRA Guide)                                                                                                                                                                                           |
| **target_modules** | All Linear Layers | This is a critical choice based on strong empirical evidence. Research shows that applying LoRA to all major linear layers—encompassing both self-attention (`q_proj`, `k_proj`, `v_proj`, `o_proj`) and the feed-forward/MLP blocks (`gate_proj`, `up_proj`, `down_proj`)—yields performance far superior to applying it only to attention layers. This ensures the adaptation happens across the model's full representational capacity. (Source: "LoRA Without Regret") |



In [ ]:
model = FastModel.get_peft_model(
    model,
    r = lora_r,
    lora_alpha = lora_r * 2, # Heuristic: 2 * r.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_dropout = 0.05, # A bit of regularization
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)


### Configuring the Trainer

The `SFTTrainer` (Supervised Fine-Tuning Trainer) requires a set of arguments to control the training process. Below is a breakdown of the key parameters used in this demonstration:

*   `per_device_train_batch_size`: The number of training examples to process on each GPU per iteration. A larger batch size can lead to more stable training but requires more memory.
*   `gradient_accumulation_steps`: This is a memory-saving technique. Instead of updating the model weights after every batch, we accumulate the gradients over multiple batches. The effective batch size becomes `per_device_train_batch_size * gradient_accumulation_steps` (in our case, 4 * 4 = 16).
*   `warmup_steps`: The learning rate starts low and gradually increases over this number of steps. This "warm-up" period helps stabilize the model at the beginning of training, preventing large, disruptive updates.
*   `max_steps`: The total number of training steps to perform. This defines the overall length of the training job.
*   `learning_rate`: The speed at which the model learns. It's a critical hyperparameter that needs to be tuned; too high, and the model might become unstable; too low, and training will be slow.
*   `logging_steps`: How frequently to print training metrics (like the loss) to the console.
*   `optim`: The optimization algorithm used to update the model's weights. `adamw_8bit` is a version of the popular AdamW optimizer that uses 8-bit precision, significantly reducing memory usage.
*   `weight_decay`: A regularization technique that penalizes large weight values, which helps prevent the model from overfitting to the training data.
*   `lr_scheduler_type`: Defines how the learning rate changes over time. A "linear" scheduler decreases the learning rate linearly from its initial value to zero over the course of training.
*   `seed`: Setting a random seed ensures that the initialization and data shuffling are the same every time you run the code, making your results reproducible.


In [ ]:
from unsloth.chat_templates import train_on_responses_only

# 1. Set up the trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text", # This is created by apply_chat_template
    args = SFTConfig(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4, # Effective batch size of 16
        warmup_steps = 10,
        max_steps = max_steps,
        learning_rate = learning_rate,
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir="outputs",
        report_to = "none"
    ),
)

# 2. Mask out user prompts for better performance
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

# 3. Start training!
trainer_stats = trainer.train()

### Inference & Evaluation

We'll now compare the performance of our fine-tuned model with the original base model.

In [ ]:
import torch

# --- 1. Inference with Fine-Tuned Model ---
prompt = "about a lighthouse keeper who befriends a whale"
messages = [
    {"role": "system", "content": "You are a master storyteller. Write a short, imaginative story based on the user's request. The story should be concise and suitable for a general audience."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = True).removeprefix('<bos>')
inputs = tokenizer(text, return_tensors = "pt").to("cuda")

# Generate output and decode, skipping the prompt
outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
finetuned_model_output = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0]

print("--- Fine-Tuned Model Output ---")
print(finetuned_model_output)

# --- 2. Inference with Base Model ---
base_model, _ = FastModel.from_pretrained(
    model_name = model_name, # Load the same base model for a fair comparison
    max_seq_length = max_seq_length,
    load_in_4bit = True,
)

# Generate output and decode, skipping the prompt
outputs = base_model.generate(**inputs, max_new_tokens = 256, use_cache = True)
base_model_output = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0]

print("\n--- Base Model Output ---")
print(base_model_output)


### Evaluation Pipeline

The diagram below shows our evaluation process. We generate responses from both the original base model and our newly fine-tuned model, then pass both outputs to a more powerful "judge" model for an objective comparison.

<div style="text-align: center;">

```mermaid
graph TD
    subgraph "Input"
        A[Test Prompt]
    end

    subgraph "Model Inference"
        A --> B[Base Gemma 3 Model];
        A --> C[Fine-Tuned Gemma 3 Model];
        B --> D[Output A: Base Response];
        C --> E[Output B: Fine-Tuned Response];
    end

    subgraph "Judgment"
        D --> F{"LLM-as-a-Judge<br>(Gemini 1.5 Flash)"};
        E --> F;
        F --> G[JSON Verdict & Comparison];
    end

    style F fill:#f9d77f,stroke:#333,stroke-width:2px
    style G fill:#a2de89,stroke:#333,stroke-width:2px
```


### LLM-as-a-Judge: The Verdict

We have the outputs, but which is better? Let's ask an impartial judge.


In [ ]:
from google import genai
import json
import os
from IPython.display import Markdown

# Ensure the Gemini API is configured for the judge model
# This is separate from the client used for the Batch API
try:
    from google.colab import userdata
    # Re-configure for non-client usage, which is simpler for single-shot generation
    client = genai.Client(api_key=google_api_key)
except (ImportError, KeyError):
    if os.environ.get("GOOGLE_API_KEY"):
         client = genai.Client(api_key=os.environ.get("GOOGLE_API_KEY"))
    else:
         print("Please set your GOOGLE_API_KEY in Colab Secrets or as an environment variable to run the judge model.")

# Use f-strings to embed the model outputs directly into the prompt
JUDGE_PROMPT = f"""
You are an expert AI model evaluator. You will be given a prompt and two stories generated by two different AI models. Your task is to analyze them and determine which one is better.

**Prompt:** {prompt}

**Story A (from Base Model):**
{base_model_output}

**Story B (from Fine-Tuned Model):**
{finetuned_model_output}

**Evaluation Criteria:**
1.  **Creativity:** How original, imaginative, and engaging is the story?
2.  **Coherence:** Is the story logical, well-structured, and easy to follow?
3.  **Adherence to Prompt:** How well does the story capture the essence of the user's request?

**Instructions:**
Provide a step-by-step analysis comparing the two stories based on the criteria above. Conclude with a final verdict in JSON format, declaring which model produced the better story and why. Provide scores from 1-10 for each criterion.

**JSON Output Format:**
```json
{{
  "winner": "Model A" or "Model B",
  "reasoning": "A brief explanation for your choice, highlighting the key differences.",
  "scores": {{
    "model_a": {{ "creativity": X, "coherence": Y, "adherence": Z }},
    "model_b": {{ "creativity": X, "coherence": Y, "adherence": Z }}
  }}
}}
```
"""

try:
    judge_model = 'gemini-2.5-flash'
    response = client.models.generate_content(
        model = judge_model,
        contents = JUDGE_PROMPT,
        config=types.GenerateContentConfig(
            temperature=0.1
        )
    )
    # show in markdown text format
    display(Markdown(response.text))
except Exception as e:
    print(f"An error occurred while calling the judge model: {e}")

# Part 4: Conclusion & Resources


### Saving, Exporting, and Sharing on the Hugging Face Hub

The final step is to save our work. This involves saving the lightweight LoRA adapters, which contain all the new knowledge learned during fine-tuning. For broader accessibility, we can also merge these adapters back into the base model and export it to various formats.

**Exporting to GGUF for Local Inference**

For local use with tools like LM Studio or Ollama, we convert the model to **GGUF (GPT-Generated Unified Format)**. This format is optimized for efficient CPU and GPU inference.

**Pushing to the Hub**

The `unsloth` library simplifies sharing on the Hugging Face Hub. To do this, we'll first save the adapters locally, then reload them to prepare for the upload. We can push multiple versions:
1.  **LoRA Adapters:** The lightweight adapter weights alone.
2.  **Merged 16-bit Model:** A full `float16` model, useful for cloud deployment with frameworks like VLLM.
3.  **GGUF Model:** The quantized model for local use.

This allows others to easily use your fine-tuned model in different environments. Ensure your Hugging Face token is configured.

In [ ]:
# Save the LoRA adapters
model.save_pretrained("gemma-3-270m-creative-writer-lora")
tokenizer.save_pretrained("gemma-3-270m-creative-writer-lora")

# You can also merge the adapters and save as a full model
# model.save_pretrained_merged("gemma-3-270m-creative-writer-merged", tokenizer, save_method = "merged_16bit")


In [ ]:
# Reload the model from the saved LoRA adapters to prepare for uploading.
from unsloth import FastModel
model, tokenizer = FastModel.from_pretrained(
    model_name = "gemma-3-270m-creative-writer-lora", # Load our saved adapters
    max_seq_length = max_seq_length,
    load_in_4bit = True,
)

In [ ]:
### Pushing to the Hugging Face Hub
if hf_user != "your-username":
    # Push the LoRA adapters
    model.push_to_hub(f"{hf_user}/gemma-3-270m-creative-writer-lora", token = hf_token)
    tokenizer.push_to_hub(f"{hf_user}/gemma-3-270m-creative-writer-lora", token = hf_token)

    # Push the merged 16-bit model
    model.push_to_hub_merged(f"{hf_user}/gemma-3-270m-creative-writer-merged", tokenizer, save_method = "merged_16bit", token = hf_token)

    # Push the GGUF model
    model.push_to_hub_gguf(f"{hf_user}/gemma-3-270m-creative-writer-gguf", tokenizer, token = hf_token)
else:
    print("Please set your Hugging Face username in the 'Notebook Configurations' cell to push to the Hub.")

### Conclusion

In under 30 minutes, we have:
1.  Understood the theory behind effective LoRA configuration.
2.  Generated a plan for a high-quality synthetic dataset.
3.  Fine-tuned a Gemma 3 model on a custom task.
4.  **Objectively evaluated the improvement** using an LLM-as-a-Judge.

This demonstrates how accessible and powerful modern AI development can be with the right tools and techniques.

### Acknowledgements & Further Reading
*   **Unsloth:** [GitHub Repository](https://github.com/unslothai/unsloth)
*   **Key Paper:** [LoRA Without Regret - Thinking Machines Lab](https://thinkingmachines.ai/blog/lora/)
*   **Gemini API:** [Batch Mode Documentation](https://ai.google.dev/gemini-api/docs/batch-mode)
